In [ ]:
import fasttext

help(fasttext.fasttext)

In [ ]:
#model_file = '/home/jvdzwaan/Downloads/wiki.ar/wiki.ar.bin'
model_file = '/media/sf_VBox_Shared/Arabic/wordembedding/wiki.ar.bin'

In [ ]:
%%time
model = fasttext.fasttext.load_model(model_file)

In [ ]:
fasttext.util.find_nearest_neighbor?

In [ ]:
%%time
from gensim.models import FastText as fText

model_gensim = fText.load_fasttext_format(model_file)

In [ ]:
import gensim
model_w2vec = gensim.models.Word2Vec.load('/media/sf_VBox_Shared/Arabic/wordembedding/wikipedia_cbow_100')

In [ ]:
def most_similar(word, topn=10):
    res = model_gensim.wv.most_similar(word, topn=topn)
    for w, s in res:
        print(u'({} {})'.format(w, s))
most_similar(u'باب')

In [ ]:
most_similar(u'سمعت')
model_w2vec.wv.most_similar(u'سمعت')

In [ ]:
most_similar(u'سمع', topn=30)
print('word2vec:')
model_w2vec.wv.most_similar(u'سمع')

In [ ]:
most_similar(u'ذوق', topn=30)
print('word2vec:')
model_w2vec.wv.most_similar(u'ذوق')

# Tune model
see: https://groups.google.com/forum/#!topic/gensim/Y_WmJST9xx8

In [ ]:
import os
import re

corpus_path = '/media/sf_VBox_Shared/Arabic/Fiqh/2018-06-08-Fiqh/'

def normalize_arabic(text):
    # Remove non-arabic characters
    nonarab_chars = '[^\u0621-\u064A ]'
    text = re.sub(nonarab_chars, '', text)
    text = text.strip()
    return text

def corpus_iterator(filepath):
    for fn in os.listdir(filepath):
        with open(os.path.join(filepath, fn)) as f:
            for line in f.readlines():
                if not line.startswith('#'):
                    words = [normalize_arabic(w) for w in line.split(' ')]
                    words = [w for w in words if len(w)>0]
                    if not len(words)==0:
                        yield words

In [ ]:
model_custom = fText.train

In [ ]:
sentences = itertools.islice(corpus_iterator(corpus_path), 100)
#sentences = [u'حدثني يحيى بن يحيى'.split(' ')]

In [ ]:
# tune fasttext: https://github.com/facebookresearch/fastText/issues/279, https://github.com/facebookresearch/fastText/pull/423
model_custom = fText(min_count=1)
#model_custom = model_gensim #. reset_from(model_gensim)
model_custom.build_vocab(sentences)
model_custom.train(sentences, epochs=1, total_examples=1)

In [ ]:
model_custom.wv.most_similar(u'يحيى')

In [ ]:
model_custom_w = gensim.models.word2vec.Word2Vec(min_count=50)
#model_custom_w.reset_from(model_w2vec)
model_custom_w.build_vocab(corpus_iterator(corpus_path))

In [ ]:
#model_custom_w.intersect_word2vec_format('/media/sf_VBox_Shared/Arabic/wordembedding/wikipedia_cbow_100')
sentences = itertools.islice(corpus_iterator(corpus_path), 100000)
model_custom_w.train(sentences, epochs=1, total_examples=model_custom_w.corpus_count)

In [ ]:
model_custom_w.most_similar(u'سمعت')